In [ ]:
# IMT 563 Winter 2021  #
# Homework 03 - Advanced SQL in Python  #
# Nicholas Marangi  #

#################################### READ ME ##################################################################################################################################################################
### The answers to these queries will different to those in the SQL page; this is primarily due to the fact that I downloaded a small subset of the total database. 
### In some cases, the Python and R answers will be subsets of the SQL equivalents; in others they will be entirely different

########## Pandas and SQL Differences
### 1. A lot of changes permanently affect the tables rather than creating temporary ones to show results like in SQL (normally not an issue but it is here), so I sometimes create a copy of each original table that would be altered by using copy()
### 2. Often when we use CTEs in SQL (at least in the problem set), we are aggregating multiple records in a group; there's quite often a groupby clause, which helps in keeping the resultant records distinct. Pandas doesn't do this, so it sometimes requires the use of drop_duplicates() to accomplish that task
##############################################################################################################################################################################################################

# Q0. Import and preparation
import numpy as np
import pandas as pd

city = pd.read_csv("../../Codework/DataSets/WWI_SubDB/City.csv")
country = pd.read_csv("../../Codework/DataSets/WWI_SubDB/Countries.csv")
custcat = pd.read_csv("../../Codework/DataSets/WWI_SubDB/CustomerCategories.csv")
cust = pd.read_csv("../../Codework/DataSets/WWI_SubDB/Customers.csv")
ordline = pd.read_csv("../../Codework/DataSets/WWI_SubDB/OrderLines.csv")
orders = pd.read_csv("../../Codework/DataSets/WWI_SubDB/Orders.csv")
peop = pd.read_csv("../../Codework/DataSets/WWI_SubDB/People.csv")
purordline = pd.read_csv("../../Codework/DataSets/WWI_SubDB/PurchaseOrderLines.csv")
purord = pd.read_csv("../../Codework/DataSets/WWI_SubDB/PurchaseOrders.csv")
st = pd.read_csv("../../Codework/DataSets/WWI_SubDB/StateProvinces.csv")
stockit = pd.read_csv("../../Codework/DataSets/WWI_SubDB/StockItems.csv")
supp = pd.read_csv("../../Codework/DataSets/WWI_SubDB/Suppliers.csv")

In [ ]:
# Q1. List the FullName, PreferredName, and number of people with the same preferred name of each person. # 

# Create NumPeopleWithPreferredName column -- DO NOT USE count() function; the combination of those below do far better
peop_temp = peop.copy()
peop_temp['NumPeopleWithPreferredName'] = peop_temp.groupby('PreferredName')['PreferredName'].transform('size')

# Order the results by FullName; Only select the attributes mentioned in the question prompt; limit the results to 30
peop_temp.sort_values('FullName')[['FullName','PreferredName','NumPeopleWithPreferredName']]

In [ ]:
# Q2. List the CustomerName, CityName, and the number of customers who share the same city, filter for all customers with 2 or more customers in the same city. # 

# Rename DeliveryCityID column in Customers to CityID; merge() needs for the 'on' parameter to have matching names
temp_cust = cust.copy()

temp_cust.rename(columns={'DeliveryCityID':'CityID'}, inplace=True)
join = temp_cust.merge(city, on = 'CityID')

# Create NumCustomersSharingCity column
join['NumCustomersSharingCity'] = join.groupby('CityID')['CustomerName'].transform('size')

# Filter by NumCustomersSharingCity being >= 2, also only select the attributes mentioned in the question
join[['CustomerName','CityName','NumCustomersSharingCity']][join['NumCustomersSharingCity'] >= 2]

In [ ]:
# Q3. List the CustomerName, CityName, StateProvinceName, CustomersPerState, LatestRecordedPopulation of the city and the ranking of each city’s population within # 
#     its given StateProvince from highest to lowest city population. # 

###### PREP & JOINS #######
city_temp = city.copy()
cust_temp = cust.copy()

city_temp.rename(columns={'LatestRecordedPopulation':'LatestRecordedCityPopulation'}, inplace=True)
cust_temp.rename(columns={'DeliveryCityID':'CityID'}, inplace=True)

# Begin joins -- inner join b/w cust and city sheds many records b/c both are subsets of the WWI database
join = cust_temp.merge(city_prep, on = 'CityID').merge(st, on = 'StateProvinceID')

# Create appropriate attributes
join['CustomersPerState'] = join.groupby("StateProvinceID")['CustomerName'].transform('size')
join['CityPopulationRank'] = join.groupby('StateProvinceID')['LatestRecordedCityPopulation'].rank(method = 'min', ascending = False)

join[['CustomerName','CityName','StateProvinceName','CustomersPerState','LatestRecordedCityPopulation','CityPopulationRank']].sort_values(['StateProvinceName','CityPopulationRank'])

In [ ]:
# Q4. List the StockItemname, UnitPrice, TypicalWeightPerUnit, and the Ranking of highest UnitPrice, returning only the top 10 stock items by rank. # 

###### PREP & JOINS #######
join = stockit.merge(ordline, on = 'StockItemID')

# There are duplicates in this that wouldn't exist in the SQL query b/c python treats the table query as SELECT * whereas Azure modifies the results based on the SQL query
# This mess of code ended up working, but I doubt that it's the optimal solution to the problem

# Selected the attributes and dropped duplicate records
join = join[['StockItemName','UnitPrice_x','TypicalWeightPerUnit']].drop_duplicates()
# Sort by UnitPrice
join = join.sort_values('UnitPrice_x', ascending = False)
# Create RankHighestUnitPrice attribute by grabbing UnitPrice and ranking it
join['RankHighestUnitPrice'] = join['UnitPrice_x'].rank(ascending = False)
# Due to the unique way this program ranks (i.e. ties are averaged), we have a significantly different result
join[join['RankHighestUnitPrice'] <= 10]

In [ ]:
# Q5. List the SalesPersonFullName, CustomerName, CityName, OrderDate, and the number of Orders that the given Salesperson was a part of for the given year (2014) # 
#     and month (6).

###### PREP & JOINS #######
peop_temp = peop.copy()
cust_temp = cust.copy()

peop_temp.rename(columns = {'PersonID':'SalespersonPersonID'}, inplace = True)
cust_temp.rename(columns={'DeliveryCityID':'CityID'}, inplace=True)


join = peop_temp.merge(orders, on = 'SalespersonPersonID').merge(cust_temp, on = 'CustomerID').merge(city, on = 'CityID')

# Filter by designated month and year -- These values return NOTHING; all of my records are from 2013, but I wanted to complete the question
join = join[(pd.DatetimeIndex(join['OrderDate']).year == 2014) & (pd.DatetimeIndex(join['OrderDate']).month == 6)]

# Create attribute OrderCount that counts the number of orders made by each Salesperson
join['OrderCount'] = join.groupby('FullName')['CustomerName'].transform('size')

# Select designated attriutes
join[['FullName','CustomerName','CityName','OrderDate','OrderCount']]

In [ ]:
# Q6. List the CustomerName, CustomerCategoryName, CreditLimit, and the rank based on the highest CreditLimit within each CustomerCategory, filtered by a rank of 5 or lower. #

###### PREP & JOINS #######

join = cust.merge(custcat, on = 'CustomerCategoryID')

# Create CreditRank attribute (na_option = bottom parameter added b/c many CreditLimit values are NULL)
join['CreditRank'] = join.groupby('CustomerCategoryName')['CreditLimit'].rank(ascending = False, na_option = 'bottom')

# Select the appropriate attributes, filter for records with a rank of 5 or lower, and sort by CategoryName and Rank
join[['CustomerName','CustomerCategoryName','CreditLimit','CreditRank']][join['CreditRank'] <= 5].sort_values(['CustomerCategoryName','CreditRank'])

In [ ]:
# Q7. List the SupplierName, StockItemName, OrderedOuters, ExpectedUnitPricePerOuter, # of Orders per Supplier and StockItem, # 
#     the sum of OrderedOuters per Supplier and StockItem, and the total spend per Supplier and StockItem.

###### PREP & JOINS ####### - 

join = supp.merge(purord, on = 'SupplierID').merge(purordline, on = 'PurchaseOrderID').merge(stockit, on = 'StockItemID')

# Create the 3 new attributes - (OrderCountPerSupplierAndStockID, SumOutersPerSupplierAndStockID, TotalSpendPerSupplierAndStockID)

join['OrderCountPerSupplerAndStockID'] = join.groupby(['SupplierID_x','StockItemID'])['PurchaseOrderID'].transform('size')
join['SumOutersPerSupplierAndStockID'] = join.groupby(['SupplierID_x','StockItemID'])['OrderedOuters'].transform('sum')

# Created the TotalPrice (per record) attribute as a stepping stone
join['TotalPrice'] = join['SumOutersPerSupplierAndStockID'] * join['ExpectedUnitPricePerOuter']
# Then sum the TotalPrices of each group
join['TotalSpendPerSupplierAndStockID'] = join.groupby(['SupplierID_x','StockItemID'])['TotalPrice'].transform('sum')

# Select needed features
join[['SupplierName','StockItemName','OrderedOuters','ExpectedUnitPricePerOuter','OrderCountPerSupplerAndStockID','SumOutersPerSupplierAndStockID','TotalSpendPerSupplierAndStockID']]

In [ ]:
# Q8. List the CountryName, StateProvinceName, CityName, and the # of cities per country and state. # 

###### PREP & JOINS #######
join = city.merge(st, on = 'StateProvinceID').merge(country, on = 'CountryID')

# Create CitiesPerCountryAndState attribute
join['CitiesPerCountryAndState'] = join.groupby(['CountryID','StateProvinceID'])['CityName'].transform('size')

#Select needed features
join[['CountryName','StateProvinceName','CityName','CitiesPerCountryAndState']].head(30)

In [ ]:
# Q9. List the CustomerName, StockItemName, TotalQuantity by Customer and StockItem, and the Rank of the highest Quantity per Customer and StockItem, #
#     filtered by a rank of 5 or lower.

###### PREP & JOINS #######
join = cust.merge(orders, on = 'CustomerID').merge(ordline, on = 'OrderID').merge(stockit, on = 'StockItemID')

# Create new attributes
join['TotalQuantityByCustomerAndStockID'] = join.groupby(['CustomerID','StockItemID'])['Quantity'].transform('sum')
join['QuantityRank'] = join.groupby('CustomerID')['TotalQuantityByCustomerAndStockID'].rank(method = 'min', ascending = False)

# Select appropriate attributes and filter for rank of <= 5; sort by CustomerName and then QuantityRank
join[['CustomerName','StockItemName','TotalQuantityByCustomerAndStockID','QuantityRank']][join['QuantityRank'] <= 5].sort_values(['CustomerName','QuantityRank'])

In [ ]:
# Q10. List the SupplierName, OrderYear, OrderMonth, TotalSpendBySupplier for a given Year and Month, and the rank by highest spend per month filtered #
#      by the ‘Farbrikam, Inc.’ supplier.

# I beleive I got this one right, but I doubt I took the optimal road to get there; having to use drop_duplicates() is what worries me

###### PREP & JOINS #######
join = supp.merge(purord, on = 'SupplierID').merge(purordline, on = 'PurchaseOrderID')

# Create most of the required attributes - added TotalSpend as a temporary/helper attribute for creating TotalSpendPerSupplierPerMonth
join['OrderYear'] = pd.DatetimeIndex(join['OrderDate']).year
join['OrderMonth'] = pd.DatetimeIndex(join['OrderDate']).month
join['TotalSpend'] = join['OrderedOuters'] * join['ExpectedUnitPricePerOuter']
join['TotalSpendPerSupplierPerMonth'] = join.groupby(['SupplierName','OrderYear','OrderMonth'])['TotalSpend'].transform('sum')

# Select the required attributes, filter to keep records with a CustomerName value containing 'Fab', drop all duplicates (created because pandas originally returns results for each individual day rather than month-year combo), and then, sort the table based on TotalSpendPerSupplierPerMonth

join = join[['SupplierName','OrderYear','OrderMonth','TotalSpendPerSupplierPerMonth']][join['SupplierName'].str.contains('Fab')].drop_duplicates().sort_values('TotalSpendPerSupplierPerMonth', ascending = False)

# Create the final attribute (Rank) here because it must occur after we drop duplicates and filter by CustomerName; otherwise, the ranking is thrown off by the additional records
join['HighestSpendPerMonth'] = join['TotalSpendPerSupplierPerMonth'].rank(ascending = False)
join


In [ ]:
# EC1. List the OrderID, OrderDate, PickingCompleteWhen, TotalOrdersForYear, TotalOrdersForYearAndMonth, the amount of days difference #
#      between the OrderDate and PickingCompletedWhen date, the amount of days difference between the previous column mentioned and the next slowest order.

###### PREP #######
orders_temp = orders.copy()

# Create required attributes - use 'next' attribute as a stepping stone to help make 'DaysGreaterThanNextSlowest'
orders_temp['TotalOrdersForYear'] = orders_temp.groupby(pd.DatetimeIndex(orders_temp['OrderDate']).year)['OrderID'].transform('size')
orders_temp['TotalOrdersForYearAndMonth'] = orders_temp.groupby([(pd.DatetimeIndex(orders_temp['OrderDate']).year),(pd.DatetimeIndex(orders_temp['OrderDate']).month)])['OrderID'].transform('size')
orders_temp['DayCountBetweenOrderAndPickup'] = (pd.to_datetime(orders_temp['PickingCompletedWhen']) - pd.to_datetime(orders_temp['OrderDate'])).dt.days
orders_temp['next'] = orders_temp['DayCountBetweenOrderAndPickup'].shift(1)
orders_temp['DaysGreaterThanNextSlowest'] = orders_temp['next'] - orders_temp['DayCountBetweenOrderAndPickup']
orders_temp['MostInconvenientRanking'] = orders_temp['DayCountBetweenOrderAndPickup'].rank(ascending = False)

# Select appropriate attributes
orders_temp[['OrderID','PickingCompletedWhen','TotalOrdersForYear','TotalOrdersForYearAndMonth','DayCountBetweenOrderAndPickup','DaysGreaterThanNextSlowest','MostInconvenientRanking']]
